In [69]:
import os
import cv2
import numpy as np
import torch
import dlib
from imutils import face_utils
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from torchvision import transforms  
import torch.nn as nn
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2
import albumentations as A
import torch.nn.functional as F
import math
import time
from imutils import face_utils
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
import mediapipe
import sys
sys.path.append("../LuminEye-Iris-Center-Localization/")
from BaseModels.resnetModels import BB_model
from BaseModels.efficientnetModels import BB_model
import pandas as pd
from scipy.spatial import distance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from scipy.spatial import distance

np.random.seed(42)

In [70]:
#Gi4e
val_csv_path = "/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/GI4E/AllCoords/gi4eAllCoordinatesTest.csv"

val_df = pd.read_csv(val_csv_path)


In [71]:
detector = None
predictor = None
GAN_MODEL = None
IRIS_MODEL = None
EYE_AR_THRESH = 0.2

mp_face_mesh = mediapipe.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)


In [72]:
def captureFaceLandmarks(frame):
    
    results = face_mesh.process(frame)
    landmarks = results.multi_face_landmarks[0]
    
    shape_arr = []
    
    for landmark in landmarks.landmark:
        
        x = landmark.x
        y = landmark.y
        
        relative_x = int(x * frame.shape[1])
        relative_y = int(y * frame.shape[0])
        
        shape_arr.append([relative_x, relative_y])
        
    
    return np.array(shape_arr)

In [73]:
# GI4E
padded_amt = 15

def cropped_image(img, shape_array, padded_amt=padded_amt ):
    """Cropped eye region 

    Args:
        img (__numpy__): _Original Image_
        shape_array (_numpy_): _FaceLandMark locations_
        padded_amt (int, optional): _padding size_. Defaults to 15.

    """

    Leye = {"top_left": shape_array[70], "bottom_right": shape_array[133]}

    Reye = {"top_left": shape_array[285],
            "bottom_right": shape_array[263]}

    left_eye = img[Leye["top_left"][1]:Leye["bottom_right"][1] +
                   padded_amt, Leye["top_left"][0]:Leye["bottom_right"][0]]

    right_eye = img[Reye["top_left"][1]:Reye["bottom_right"][1] +
                    padded_amt, Reye["top_left"][0]:Reye["bottom_right"][0]]

    return left_eye, right_eye, Leye, Reye

In [74]:
def mpArrayToNumpy(landmark_array, img):

    shape_arr = []

    for landmark in landmark_array.landmark:
        x = landmark.x
        y = landmark.y

        relative_x = int(x * img.shape[1])
        relative_y = int(y * img.shape[0])

        shape_arr.append([relative_x, relative_y])

    return np.array(shape_arr)

In [75]:
#GI4E
IMG_DIR = "/home/nipun/Documents/Uni_Malta/Datasets/gi4e_database/images/"

# histogramPath

histogramPath = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-MainPipeLine/BayesClassifier/MatLab/ScleraSkinIris.txt"


In [76]:
def BayesEstimation(img,histogramBinValuesPath,thresh=1.0):
    
    r_img = img[:,:,::-1][:,:,0]
    
    imgGauss = cv2.GaussianBlur(r_img,(3,3),1)
    
    with open(histogramBinValuesPath,"r") as f:
        hist = [float(j.rstrip()) for j in f.readlines()]
        
    imgR_bin = np.zeros((r_img.shape[0],r_img.shape[1])).astype(np.uint8)
    
    
    
    for row in range(r_img.shape[0]):
    
        for column in range(r_img.shape[1]):
            
            binNo = (imgGauss[row,column]) // 4
            
            
            # print(binNo)
            likelihood = hist[binNo]
            
            
            if likelihood >= thresh:
                imgR_bin[row,column] = 1
                
                
    # plt.imshow(imgR_bin)
    # plt.show()
    
    
    
    analysis = cv2.connectedComponentsWithStats(imgR_bin,
                                           4,
                                            cv2.CV_32S)
    (totalLabels, label_ids, values, centroid) = analysis
    
    
    return centroid[-1]

In [77]:
# GI4E
count = 0

maximizedNormalizedError = []
for _, row in val_df.iterrows():

    img_name = row["ImageName"]

    coords = [float(j) for j in row["Coordinates"][1:-1].split(",")]

    left_center = coords[2:4]

    right_center = coords[8:10]

    IPD = distance.euclidean(left_center, right_center)

    image_filepath = os.path.join(IMG_DIR, img_name)

    img = cv2.imread(image_filepath)

    results = face_mesh.process(img)

    if results.multi_face_landmarks is not None:
        landmarks = results.multi_face_landmarks[0]

        shape_arr = mpArrayToNumpy(landmarks, img)
        
        
        cv2.circle(img, (int(left_center[0]), int(
            left_center[1])), 1, (0, 0, 255), -1)

        cv2.circle(img, (int(right_center[0]), int(
            right_center[1])), 1, (0, 0, 255), -1)

        left_eye, right_eye, Leye, Reye = cropped_image(img, shape_arr)

        left_centroid = BayesEstimation(left_eye, histogramPath)
        right_centroid = BayesEstimation(right_eye, histogramPath)

        cv2.circle(left_eye, (int(left_centroid[0]), int(
            left_centroid[1])), 1, (0, 255, 0), -1)

        cv2.circle(right_eye, (int(right_centroid[0]), int(
            right_centroid[1])), 1, (0, 255, 0), -1)

        pred_l_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            left_centroid, Leye)
        pred_r_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            right_centroid, Reye)

        cv2.circle(img, (int(pred_l_eye_toOriginaImage[0]), int(
            pred_l_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)

        cv2.circle(img, (int(pred_r_eye_toOriginaImage[0]), int(
            pred_r_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)

        

        left_eye_euclidea_distance = distance.euclidean(
            pred_l_eye_toOriginaImage, left_center)

        right_eye_euclidea_distance = distance.euclidean(
            pred_r_eye_toOriginaImage, right_center)

        eMax = max(left_eye_euclidea_distance, right_eye_euclidea_distance)/IPD

        print(eMax)

        maximizedNormalizedError.append(eMax)

        # fig, axes = plt.subplots(1, 3)

        # fig.set_figwidth(15)
        # fig.set_figheight(15)

        # axes[0].imshow(img[:, :, ::-1])
        # axes[1].imshow(left_eye[:, :, ::-1])
        # axes[2].imshow(right_eye[:, :, ::-1])

        # axes[0].axis("off")
        # axes[1].axis("off")
        # axes[2].axis("off")

        # plt.tight_layout()
        # plt.show()

        # count += 1

        # if count == 100:
        #     break

    else:

        print(
            f"MediaPipe was failed to detect the faces on the image name {obj[0]}")
        continue




0.023419807430501153
0.24412427173355677
0.25280572233834075
0.04030926480727125
0.1972439634785642
0.22385994312424817
0.17644547895595472
0.2511572189725823
0.27784102530451127
0.05379816560691199
0.18583363518342094
0.15589545356860074
0.22312975576652733
0.2183264347392754
0.2098084760156023
0.18276525375414104
0.04872230452998477
0.24202154973881632
0.06094466840906249
0.06806315224431106
0.21794166007963728
0.06897666899746413
0.06557612285353298
0.059197329019360805
0.04225895489061842
0.07549741980324659
0.28145118207893083
0.04775746399327072
0.04638956136953164
0.233973697844669
0.20336617342322905
0.16481127863103756
0.13404955619633774
0.05898288280700795
0.24715115695084192
0.03273421139122169
0.19750666086748897
0.04349568430037221
0.034546770511860794
0.18710768686396806
0.03629733262404816
0.1496027742064993
0.02471109696907519
0.20298491949145372
0.17572643855504144
0.2145317569040024
0.04131716060729893
0.038260418824743604
0.2323509145562091
0.15161801727663043
0.183

In [78]:
def scaleCoorinatesToOriginalImage(pred_coords,eye_margin):
    
    # {'top_left': array([385, 214]), 'bottom_right': array([426, 226])}
    
    x1 = eye_margin["top_left"][0] + pred_coords[0]
    y1 = eye_margin["top_left"][1] + pred_coords[1]
    
    return [x1,y1]

In [79]:
def calculateEuclideanDistance(coord1,coord2):
    return (((coord1[0])-float(coord2[0]))^2  + (float(coord1[0])-float(coord2[0]))^2) ^ 0.5

In [80]:
def CheckForLess(list1, val):
    
    l1 = []
    for x in list1:
        
        if x <= val:
            l1.append(x)
    return l1

In [81]:
e_0_25 = CheckForLess(maximizedNormalizedError,0.25)

e_0_05 = CheckForLess(maximizedNormalizedError,0.05)

e_0_1 = CheckForLess(maximizedNormalizedError,0.1)

In [82]:
(len(e_0_25)/len(maximizedNormalizedError)) *100

91.22807017543859

In [83]:
(len(e_0_05)/len(maximizedNormalizedError)) * 100

15.350877192982457

In [84]:
len(e_0_1)/len((maximizedNormalizedError)) * 100

36.84210526315789